### Topic Modeling with NNMF

In [1]:
import pandas as pd
import re

In [2]:
# import modern magic cards from file 
modern = pd.read_pickle('data/5color_modern_no_name_hardmode.pkl')

def nolist(x):
    return x[-1]

modern['type'] = modern['types'].apply(nolist)

modern.head(2)

,artist,cmc,colors,flavor,manaCost,name,power,rarity,text,toughness,type,types,set,releaseDate
0,Michael Sutfin,4,Black,To gaze under its hood is to invite death.,{2}{B}{B},Abyssal Specter,2,Uncommon,"Flying Whenever This deals damage to a player,...",3,Creature,[Creature],Eighth Edition,2003-07-28
1,Wayne England,5,Blue,Pray that it doesn't seek the safety of your l...,{3}{U}{U},Air Elemental,4,Uncommon,Flying,4,Creature,[Creature],Eighth Edition,2003-07-28


In this case we have too much data. No problems with unlabeled or missing data. 

#### CountVectorizer Time 
Starting simple. Just a term frequency matrix. 

In [3]:
# vocab size 
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words="english")
vectorized_data = vectorizer.fit_transform(modern.text) 

print "There are {:,} words in the vocabulary.".format(len(vectorizer.vocabulary_))

There are 1,020 words in the vocabulary.


#### Non-negative matrix factorization 

In [23]:
# Non-Negative Matrix factorization

# code modified from documentation

from sklearn.decomposition import NMF

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print
        print "Topic #%d:" % (topic_idx + 1)
        print " ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]])
    
    
nmf = NMF(n_components=9, random_state=41).fit(vectorized_data)

nmf_feature_names = vectorizer.get_feature_names()

print_top_words(nmf, nmf_feature_names, 20)


Topic #1:
creature sacrifice token dies power green white blocked black attacks battlefield tokens deals control damage blocks attacking additional zombie regenerate

Topic #2:
card library cards hand graveyard player search reveal draw shuffle mana land exile cost puts cast return revealed reveals look

Topic #3:
damage player deals combat dealt flying prevent turn target equal deal controls instead cards number source way players discards prevented

Topic #4:
turn end gets gains flying haste trample gain strike beginning cast dealt target attacks creatures activate ability prevent able step

Topic #5:
battlefield enters flying token return counter tokens green counters land white control exile graveyard sacrifice tapped artifact haste leaves owners

Topic #6:
control creatures number power gain equal long opponents tap flying toughness counter thiss dont lands untap owners greater trample end

Topic #7:
life gain loses player opponent equal beginning upkeep lose total pay cards toug

In [7]:
H = nmf.transform(vectorized_data)
W = nmf.components_
print H.shape
print W.shape

(7874L, 9L)
(9L, 1020L)


Here we have our H matrix of 7874 individual cards, and 9 lables.  
and in the W matrix we have 9 lables, we have 1020 unique words in the vocabulary.  

In [22]:
# modified code from documentation

def print_top_cards(model, feature_names, n_top_art):
    human_lables = [' enchant ', ' deck manipulation ', ' damage ', 
                    ' planewalkers/choose one ', ' creature tokens ', 
                    ' counters ', 
                    ' life total tricks ', '---- ', ' enchant ']
    for topic_idx, topic in enumerate(model.T):
        print "Topic: -----------%s---------" % human_lables[topic_idx]
        indices = [i for i in topic.argsort()[:-n_top_art - 1:-1]]
        for i in indices:
            print "Actual color =", modern.colors[i]
            print "Actual type =", modern.type[i]
            print modern.text[i][:158]
            print
        print
        print

    print
    
print_top_cards(H, nmf_feature_names, 4)

Topic: ----------- enchant ---------
Actual color = Green
Actual type = Enchantment
Enchant creature Enchanted creature has "Tap : This creature deals damage equal to its power to target creature. That creature deals damage equal to its power

Actual color = Black
Actual type = Enchantment
Enchant creature Enchanted creature has "At the beginning of your upkeep, sacrifice this creature." When enchanted creature dies, its controller chooses targe

Actual color = Green
Actual type = Enchantment
Enchant creature Enchanted creature gets +2/+2. When enchanted creature dies, you may return This from your graveyard to the battlefield attached to a creatur

Actual color = Black
Actual type = Instant
Destroy target nonwhite, nonblack creature. Put a 1/1 white Spirit creature token with flying onto the battlefield. Haunt  When the creature This haunts dies,



Topic: ----------- deck manipulation ---------
Actual color = Black
Actual type = Sorcery
Reveal a card from your hand. Search your libra

Most common missclassifications are everything. Working from a data munged set of just the 5 colors, expecting 5 colors back out. What I got out were rules topics, not colors topics. Different types of cards, and actions you can take in the game.  

### To test: lets throw in all the cards. 

Previous dataset had colorless and multicolor cards removed.

In [24]:
# import all modern cards 
all_modern = pd.read_pickle('data/all_cards_modern_no_name.pkl')

# vectorize 
vectorizer = CountVectorizer(stop_words='english')
vectorized_data = vectorizer.fit_transform(all_modern.text) 

print "There are {:,} words in the vocabulary.".format(len(vectorizer.vocabulary_))

# NMF
nmf = NMF(n_components=9, random_state=41).fit(vectorized_data)

nmf_feature_names = vectorizer.get_feature_names()

print_top_words(nmf, nmf_feature_names, 20)

There are 5,723 words in the vocabulary.

Topic #1:
creature enchanted enchant target gets equip flying equipped tap sacrifice token destroy blocked untap long dies attached attach block counter

Topic #2:
card graveyard hand draw target return exile cost player discard opponent library reveals converted face discards reveal beginning owners sorcery

Topic #3:
turn end gets target gains haste gain attacks trample dealt strike ability activate beginning step flying able untap face blocks

Topic #4:
damage player deals target combat life dealt destroy equal controls creature counters choose tap opponent prevent players cards deal defending

Topic #5:
battlefield enters counter token control counters tapped life return land sacrifice owners beginning flying green gain tokens leaves opponent permanent

Topic #6:
mana tap pool add color tapped land sacrifice untap charge converted cost activate enters life spend remove ability colorless pay

Topic #7:
control creatures flying gain blocked p

In [25]:
print_top_cards(H, nmf_feature_names, 4)

Topic: ----------- enchant ---------
Actual color = Green
Actual type = Enchantment
Enchant creature Enchanted creature has "Tap : This creature deals damage equal to its power to target creature. That creature deals damage equal to its power

Actual color = Black
Actual type = Enchantment
Enchant creature Enchanted creature has "At the beginning of your upkeep, sacrifice this creature." When enchanted creature dies, its controller chooses targe

Actual color = Green
Actual type = Enchantment
Enchant creature Enchanted creature gets +2/+2. When enchanted creature dies, you may return This from your graveyard to the battlefield attached to a creatur

Actual color = Black
Actual type = Instant
Destroy target nonwhite, nonblack creature. Put a 1/1 white Spirit creature token with flying onto the battlefield. Haunt  When the creature This haunts dies,



Topic: ----------- deck manipulation ---------
Actual color = Black
Actual type = Sorcery
Reveal a card from your hand. Search your libra

Results:
- Most common missclassifications are everything. Working from a data munged set of just the 5 colors, expecting 5 colors back out. What I got out were rules topics, not colors topics. Different types of cards, and actions you can take in the game.  To test lets throw in all the cards.  
- Topics can be assigned to a few categories. They are labeled above. 
- Little to no change when my carefully data munged cards were put back in. Hypothesis confirmed. Topic modeling is not affected by color. Topic modeling is finding something else to group by.  
- English stop words removed.

### LDA visualization with GraphLab

In [14]:
## pyLDAvis copy/paste

from __future__ import absolute_import

import funcy as fp
import numpy as np
import pandas as pd
import graphlab as gl
import pyLDAvis

def _topics_as_df(topic_model):
    tdf = topic_model['topics'].to_dataframe()
    return pd.DataFrame(np.vstack(tdf['topic_probabilities'].values), index=tdf['vocabulary'])

def _sum_sarray_dicts(sarray):
    counts_sf = gl.SFrame({'count_dicts': sarray}).stack('count_dicts').groupby(key_columns='X1',
                                              operations={'count': gl.aggregate.SUM('X2')})
    return counts_sf.unstack(column=['X1', 'count'])[0].values()[0]

def _extract_doc_data(docs):
    doc_lengths = list(docs.apply(lambda d: np.array(d.values()).sum()))
    term_freqs_dict = _sum_sarray_dicts(docs)

    vocab = term_freqs_dict.keys()
    term_freqs = term_freqs_dict.values()

    return {'doc_lengths': doc_lengths, 'vocab': vocab, 'term_frequency': term_freqs}

def _extract_model_data(topic_model, docs, vocab):
    doc_topic_dists = np.vstack(topic_model.predict(docs, output_type='probabilities'))

    topics = _topics_as_df(topic_model)
    topic_term_dists = topics.T[vocab].values

    return {'topic_term_dists': topic_term_dists, 'doc_topic_dists': doc_topic_dists}

def _extract_data(topic_model, docs):
    doc_data = _extract_doc_data(docs)
    model_data = _extract_model_data(topic_model, docs, doc_data['vocab'])
    return fp.merge(doc_data, model_data)

def prepare(topic_model, docs, **kargs):
    """Transforms the GraphLab TopicModel and related corpus data into
    the data structures needed for the visualization.
    Parameters
    ----------
    topic_model : graphlab.toolkits.topic_model.topic_model.TopicModel
        An already trained GraphLab topic model.
    docs : SArray of dicts
        The corpus in bag of word form, the same docs used to train the model.
    **kwargs :
        additional keyword arguments are passed through to :func:`pyldavis.prepare`.
    Returns
    -------
    prepared_data : PreparedData
        the data structures used in the visualization
    Example
    --------
    For example usage please see this notebook:
    http://nbviewer.ipython.org/github/bmabey/pyLDAvis/blob/master/notebooks/GraphLab.ipynb
    """
    opts = fp.merge(_extract_data(topic_model, docs), kargs)
    return pyLDAvis.prepare(**opts)

In [13]:
# LDA vis

import graphlab as gl
import pyLDAvis 

pyLDAvis.enable_notebook()

modernSFrame = gl.SFrame(modern[["text", "colors"]])

# modernSFrame.show()

modernSFrame['features'] = gl.text_analytics.count_ngrams(modernSFrame['text'], 1)

topicModel = gl.topic_model.create(modernSFrame['features'], num_topics=14, num_iterations=50)

prepare(topicModel, modernSFrame['features'])



PROGRESS: Learning a topic model
PROGRESS:        Number of documents      7874
PROGRESS:            Vocabulary size      1299
PROGRESS:    Running collapsed Gibbs sampling
PROGRESS: +-----------+---------------+----------------+-----------------+
PROGRESS: | Iteration | Elapsed Time  | Tokens/Second  | Est. Perplexity |
PROGRESS: +-----------+---------------+----------------+-----------------+
PROGRESS: | 10        | 384.863ms     | 4.33239e+06    | 0               |
PROGRESS: | 20        | 732.8ms       | 4.52632e+06    | 0               |
PROGRESS: | 30        | 1.06s         | 4.37516e+06    | 0               |
PROGRESS: | 40        | 1.41s         | 4.70569e+06    | 0               |
PROGRESS: | 50        | 1.76s         | 4.22612e+06    | 0               |
PROGRESS: +-----------+---------------+----------------+-----------------+


PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
9      7.659618        1       1 -0.248999  0.134495
5      7.581508        1       2 -0.189961 -0.236728
7      7.579246        1       3 -0.130133 -0.296649
2      7.471678        1       4  0.124872 -0.148886
3      7.287821        1       5 -0.191557 -0.107989
4      7.160769        1       6 -0.169482  0.211220
13     7.047415        1       7  0.017795  0.164610
8      7.036409        1       8  0.154000 -0.178163
11     7.011962        1       9  0.157780  0.008496
0      6.909925        1      10  0.087494 -0.033548
12     6.892207        1      11 -0.197572  0.215390
1      6.842584        1      12  0.186696  0.091678
6      6.834671        1      13  0.204245  0.091029
10     6.684187        1      14  0.194823  0.085046, topic_info=     Category         Freq           Term  Total  loglift  logprob
266   Default  7084.000000              1   7084  30.0000  30.0000
1160  Default  3441.000000           your   3441  29.0000  29.0000
1135  Default  5248.000000            you   5248  28.0000  28.0000
964   Default  5461.000000            the   5461  27.0000  27.0000
221   Default  7367.000000           this   7367  26.0000  26.0000
277   Default  6495.000000       creature   6495  25.0000  25.0000
232   Default  4286.000000             of   4286  24.0000  24.0000
1057  Default  2425.000000    battlefield   2425  23.0000  23.0000
991   Default  3615.000000             to   3615  22.0000  22.0000
265   Default  1874.000000         damage   1874  21.0000  21.0000
1141  Default  2837.000000             or   2837  20.0000  20.0000
465   Default  2347.000000           turn   2347  19.0000  19.0000
535   Default  3582.000000         target   3582  18.0000  18.0000
301   Default  4658.000000              a   4658  17.0000  17.0000
889   Default  2945.000000             it   2945  16.0000  16.0000
241   Default  2136.000000        control   2136  15.0000  15.0000
259   Default  1640.000000            end   1640  14.0000  14.0000
304   Default  1495.000000          until   1495  13.0000  13.0000
455   Default  2586.000000           card   2586  12.0000  12.0000
1231  Default  1357.000000             as   1357  11.0000  11.0000
872   Default  1449.000000           cast   1449  10.0000  10.0000
505   Default  1983.000000              2   1983   9.0000   9.0000
891   Default  1718.000000         player   1718   8.0000   8.0000
89    Default  1756.000000      creatures   1756   7.0000   7.0000
994   Default  1286.000000         enters   1286   6.0000   6.0000
62    Default  1201.000000          deals   1201   5.0000   5.0000
1116  Default  1447.000000             on   1447   4.0000   4.0000
285   Default  1650.000000            may   1650   3.0000   3.0000
974   Default  1354.000000           when   1354   2.0000   2.0000
892   Default  1012.000000           cant   1012   1.0000   1.0000
...       ...          ...            ...    ...      ...      ...
1037  Topic14    13.059141           lost     16   2.6147  -6.7227
424   Topic14    23.189612              p     29   2.5561  -6.1867
724   Topic14    12.091092  constellation     15   2.6057  -6.7963
198   Topic14    70.132011           type    100   2.3859  -5.1190
986   Topic14    33.454179          human     45   2.4638  -5.8396
548   Topic14   290.214425       opponent    504   2.1784  -3.7090
1253  Topic14   913.687334           that   1865   2.0126  -2.5664
535   Topic14  1603.149235         target   3582   1.9100  -2.0164
321   Topic14    28.505638          hands     40   2.4230  -5.9981
654   Topic14    66.774033           much    106   2.2689  -5.1777
891   Topic14   677.438068         player   1718   1.7922  -2.8689
1212  Topic14    30.679627           flip     46   2.3497  -5.9317
658   Topic14   167.534331       controls    344   2.0071  -4.2623
850   Topic14   111.829530     controller    223   2.0444  -4.6584
1189  Topic14   110.239817 

In [33]:
from collections import Counter

Counter(modern['type'])

Counter({u'Artifact': 23,
         u'Creature': 4400,
         u'Enchantment': 972,
         u'Instant': 1321,
         u'Planeswalker': 59,
         u'Sorcery': 1099})

In [34]:
modernSFrame2 = gl.SFrame(modern[["text", "type"]])

# modernSFrame.show()

modernSFrame2['features'] = gl.text_analytics.count_ngrams(modernSFrame2['text'], 1)

topicModel2 = gl.topic_model.create(modernSFrame2['features'], num_topics=6, num_iterations=50)

prepare(topicModel2, modernSFrame2['features'])

PROGRESS: Learning a topic model
PROGRESS:        Number of documents      7874
PROGRESS:            Vocabulary size      1299
PROGRESS:    Running collapsed Gibbs sampling
PROGRESS: +-----------+---------------+----------------+-----------------+
PROGRESS: | Iteration | Elapsed Time  | Tokens/Second  | Est. Perplexity |
PROGRESS: +-----------+---------------+----------------+-----------------+
PROGRESS: | 10        | 407.097ms     | 3.51787e+06    | 0               |
PROGRESS: | 20        | 785.577ms     | 3.77455e+06    | 0               |
PROGRESS: | 30        | 1.16s         | 4.03943e+06    | 0               |
PROGRESS: | 40        | 1.55s         | 3.46689e+06    | 0               |
PROGRESS: | 50        | 1.93s         | 3.87689e+06    | 0               |
PROGRESS: +-----------+---------------+----------------+-----------------+


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      17.454870        1       1  0.198856 -0.025405
4      17.217563        1       2  0.227909 -0.067814
5      16.645885        1       3  0.046623  0.243514
3      16.543466        1       4 -0.105082  0.209789
0      16.165968        1       5 -0.024631 -0.290094
1      15.972248        1       6 -0.343675 -0.069990, topic_info=     Category         Freq         Term  Total  loglift  logprob
266   Default  7084.000000            1   7084  30.0000  30.0000
221   Default  7367.000000         this   7367  29.0000  29.0000
277   Default  6495.000000     creature   6495  28.0000  28.0000
1160  Default  3441.000000         your   3441  27.0000  27.0000
964   Default  5461.000000          the   5461  26.0000  26.0000
1135  Default  5248.000000          you   5248  25.0000  25.0000
535   Default  3582.000000       target   3582  24.0000  24.0000
301   Default  4658.000000            a   4658  23.0000  23.0000
991   Default  3615.000000           to   3615  22.0000  22.0000
232   Default  4286.000000           of   4286  21.0000  21.0000
1141  Default  2837.000000           or   2837  20.0000  20.0000
465   Default  2347.000000         turn   2347  19.0000  19.0000
304   Default  1495.000000        until   1495  18.0000  18.0000
340   Default  1608.000000          its   1608  17.0000  17.0000
1170  Default  1453.000000       flying   1453  16.0000  16.0000
259   Default  1640.000000          end   1640  15.0000  15.0000
1231  Default  1357.000000           as   1357  14.0000  14.0000
1057  Default  2425.000000  battlefield   2425  13.0000  13.0000
455   Default  2586.000000         card   2586  12.0000  12.0000
241   Default  2136.000000      control   2136  11.0000  11.0000
62    Default  1201.000000        deals   1201  10.0000  10.0000
505   Default  1983.000000            2   1983   9.0000   9.0000
89    Default  1756.000000    creatures   1756   8.0000   8.0000
872   Default  1449.000000         cast   1449   7.0000   7.0000
1112  Default  1355.000000         from   1355   6.0000   6.0000
889   Default  2945.000000           it   2945   5.0000   5.0000
920   Default  1114.000000         gets   1114   4.0000   4.0000
891   Default  1718.000000       player   1718   3.0000   3.0000
974   Default  1354.000000         when   1354   2.0000   2.0000
452   Default  1742.000000          put   1742   1.0000   1.0000
...       ...          ...          ...    ...      ...      ...
341    Topic6    66.500063       before     67   1.8242  -6.0811
358    Topic6    47.501826         ally     48   1.8248  -6.4140
31     Topic6   441.327177            0    474   1.7530  -4.1958
665    Topic6    41.502709            8     42   1.8251  -6.5473
792    Topic6    40.502881        comes     41   1.8251  -6.5713
21     Topic6    61.503033       causes     63   1.8083  -6.1586
576    Topic6    33.504364         once     34   1.8256  -6.7580
634    Topic6   572.629298         gain    624   1.7376  -3.9363
449    Topic6    27.506212         base     28   1.8263  -6.9515
1107   Topic6   262.281054         step    281   1.7569  -4.7148
1168   Topic6    97.502344    artifacts    102   1.7837  -5.7013
672    Topic6    22.508470      prowess     23   1.8270  -7.1475
1098   Topic6   368.693328        power    411   1.7152  -4.3762
405    Topic6   136.452080     activate    147   1.7530  -5.3666
465    Topic6  1843.558084         turn   2347   1.5832  -2.7659
206    Topic6   345.463383        under    400   1.6780  -4.4406
1121   Topic6   225.016888      attacks    256   1.6985  -4.8663
244    Topic6   796.413718          tap   1024   1.5760  -3.6026
686    Topic6   136.606701       during    151   1.7261  -5.3666
221    Topic6  4307.068819         this   7367   1.2843  -1.9209
784    Topic6   281.281832        equal    371   1.5488  -4.6451
232    Topic6  1966.384822           of   4286   1.0515  -2.6954
535    Topic6  